# 대구광역시 역세권 거주인구 추정 및 향후 변화 예측(GBT, SVR 예측 모델, 엄선현)

In [1]:
import pandas as pd
import numpy as np
import os
from keras.models import Sequential 
from keras.layers import Dense 
from keras.wrappers.scikit_learn import KerasClassifier 
from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

Using TensorFlow backend.


In [2]:
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import mean_absolute_error

In [3]:
pd.options.display.max_columns = 100

In [4]:
os.getcwd()

'/Users/um/Desktop/취업/git-hub/Station_Influence _Area_ prediction'

In [6]:
os.chdir("/Users/um/Desktop/2020년/2학기/(2020-2)공무원프로젝트/data/3가지 예측모델 데이터")
os.getcwd()

'/Users/um/Desktop/2020년/2학기/(2020-2)공무원프로젝트/data/3가지 예측모델 데이터'

## 클러스터링 데이터

In [7]:
data = pd.read_excel("20201117_전처리.xlsx", sheet_name="원본")

In [8]:
data

,Month,역명,가설건축물,공공용시설,공동주택,공장,관광휴게시설,교육연구및복지시설,교육연구시설,교정및군사시설,노유자시설,단독주택,동.식물관련시설,묘지관련시설,문화및집회시설,발전시설,방송통신시설,분뇨.쓰레기처리시설,수련시설,숙박시설,업무시설,운동시설,운수시설,위락시설,위험물저장및처리시설,의료시설,자동차관련시설,자원순환관련시설,장례시설,제1종근린생활시설,제2종근린생활시설,종교시설,창고시설,판매및영업시설,판매시설,05~06,06~07,07~08,08~09,09~10,10~11,11~12,12~13,13~14,14~15,15~16,16~17,17~18,18~19,19~20,20~21,21~22,22~23,23~24,2019승객수,2020총수송인구수,cluster
0,1,각산,0,0,38,18,1,5,18,0,2,405,0,0,9,0,0,0,0,0,20,0,0,0,3,0,26,0,0,97,135,2,17,4,12,1415.0,5457.0,12550.0,20776.0,15592.0,12780.0,14494.0,15192.0,17643.0,15839.0,17664.0,17077.0,20291.0,24557.0,15780.0,10670.0,10848.0,10228.0,5464.0,264317.0,260249.0,7
1,10,각산,0,0,38,18,1,5,18,0,2,409,0,0,9,0,0,0,0,0,21,0,0,0,3,0,26,0,0,98,138,2,19,4,19,1932.0,6702.0,17718.0,23870.0,17953.0,13577.0,14894.0,16102.0,16143.0,16666.0,17309.0,18739.0,21736.0,25224.0,16863.0,11291.0,11348.0,9817.0,5452.0,283336.0,211568.0,7
2,2,각산,0,0,38,18,1,0,18,0,2,407,0,0,3,0,0,0,0,0,20,0,0,0,3,0,26,0,0,98,135,2,17,0,12,1134.0,4430.0,10554.0,16825.0,13250.0,11471.0,12714.0,13539.0,15349.0,14142.0,15331.0,15158.0,18424.0,20985.0,14106.0,10062.0,9790.0,9013.0,5485.0,231762.0,176565.0,7
3,3,각산,0,0,38,18,1,0,18,0,2,409,0,0,3,0,0,0,0,0,20,0,0,0,3,0,26,0,0,98,136,2,17,0,12,1510.0,5954.0,17813.0,21512.0,16471.0,12708.0,14140.0,14941.0,16102.0,14791.0,16861.0,17892.0,21551.0,24640.0,17131.0,11258.0,11796.0,10831.0,5940.0,273842.0,88349.0,7
4,4,각산,0,0,38,18,1,5,18,0,2,409,0,0,9,0,0,0,0,0,20,0,0,0,3,0,26,0,0,98,136,2,17,4,12,1760.0,6520.0,19021.0,23846.0,17862.0,12508.0,13092.0,15656.0,15709.0,13641.0,16556.0,17866.0,20117.0,23918.0,16591.0,11301.0,10938.0,10072.0,5502.0,272476.0,127276.0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
845,5,황금,0,0,17,0,1,5,9,0,11,391,0,0,8,0,6,0,0,44,6,6,1,35,1,3,6,0,0,122,404,3,8,0,5,1303.0,3848.0,9247.0,14914.0,13277.0,11303.0,12143.0,13855.0,13900.0,14807.0,15190.0,16322.0,16146.0,19205.0,12789.0,9079.0,8055.0,6383.0,4202.0,215968.0,133157.0,3
846,6,황금,0,0,17,0,1,5,9,0,11,392,0,0,8,0,6,0,0,44,6,6,1,35,1,3,6,0,0,122,405,3,8,0,5,1405.0,3531.0,8130.0,13736.0,12062.0,10545.0,11529.0,12710.0,12760.0,12975.0,13252.0,14484.0,14912.0,17446.0,11354.0,8371.0,7693.0,6116.0,3687.0,196698.0,142248.0,3
847,7,황금,0,0,17,0,1,5,9,0,11,393,0,0,8,0,6,0,0,44,6,6,1,35,1,3,7,0,0,122,406,3,8,0,5,1280.0,3650.0,8053.0,14965.0,12426.0,11610.0,12313.0,13764.0,13833.0,13509.0,13662.0,15225.0,15740.0,18944.0,12120.0,8668.0,7973.0,6701.0,3848.0,208284.0,149504.0,3
848,8,황금,0,0,17,0,1,5,9,0,11,394,0,0,8,0,6,0,0,44,6,6,1,35,1,3,7,0,0,122,411,3,8,0,5,1272.0,3320.0,7189.0,13044.0,12126.0,11076.0,11433.0,12581.0,12703.0,12515.0,12774.0,14157.0,14692.0,17487.0,11205.0,7991.0,7285.0,6217.0,4016.0,193083.0,137694.0,3


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 850 entries, 0 to 849
Data columns (total 57 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Month       850 non-null    int64  
 1   역명          850 non-null    object 
 2   가설건축물       850 non-null    int64  
 3   공공용시설       850 non-null    int64  
 4   공동주택        850 non-null    int64  
 5   공장          850 non-null    int64  
 6   관광휴게시설      850 non-null    int64  
 7   교육연구및복지시설   850 non-null    int64  
 8   교육연구시설      850 non-null    int64  
 9   교정및군사시설     850 non-null    int64  
 10  노유자시설       850 non-null    int64  
 11  단독주택        850 non-null    int64  
 12  동.식물관련시설    850 non-null    int64  
 13  묘지관련시설      850 non-null    int64  
 14  문화및집회시설     850 non-null    int64  
 15  발전시설        850 non-null    int64  
 16  방송통신시설      850 non-null    int64  
 17  분뇨.쓰레기처리시설  850 non-null    int64  
 18  수련시설        850 non-null    int64  
 19  숙박시설        850 non-null    i

In [17]:
test_name = ["대명","두류","대곡(정부대구청사)","학정","문양","용계","동대구역","반월당","어린이회관","칠성시장","큰고개","사월","계명대","영대병원","중앙로"
            ,"범물","용산(서부법원?검찰청입구)","대구은행","건들바위","아양교","남산","서문시장(동산병원)","상인","대구역"]
test_name

['대명',
 '두류',
 '대곡(정부대구청사)',
 '학정',
 '문양',
 '용계',
 '동대구역',
 '반월당',
 '어린이회관',
 '칠성시장',
 '큰고개',
 '사월',
 '계명대',
 '영대병원',
 '중앙로',
 '범물',
 '용산(서부법원?검찰청입구)',
 '대구은행',
 '건들바위',
 '아양교',
 '남산',
 '서문시장(동산병원)',
 '상인',
 '대구역']

In [18]:
len(test_name)

24

In [19]:
test_data = data[data["역명"].isin(test_name)]

In [20]:
test_data["역명"]

30     건들바위
31     건들바위
32     건들바위
33     건들바위
34     건들바위
       ... 
805      학정
806      학정
807      학정
808      학정
809      학정
Name: 역명, Length: 240, dtype: object

In [21]:
train_data = data[data["역명"].isin(test_name)==False]

In [22]:
train_data["역명"]

0      각산
1      각산
2      각산
3      각산
4      각산
       ..
845    황금
846    황금
847    황금
848    황금
849    황금
Name: 역명, Length: 610, dtype: object

In [23]:
data.columns

Index(['Month', '역명', '가설건축물', '공공용시설', '공동주택', '공장', '관광휴게시설', '교육연구및복지시설',
       '교육연구시설', '교정및군사시설', '노유자시설', '단독주택', '동.식물관련시설', '묘지관련시설', '문화및집회시설',
       '발전시설', '방송통신시설', '분뇨.쓰레기처리시설', '수련시설', '숙박시설', '업무시설', '운동시설', '운수시설',
       '위락시설', '위험물저장및처리시설', '의료시설', '자동차관련시설', '자원순환관련시설', '장례시설',
       '제1종근린생활시설', '제2종근린생활시설', '종교시설', '창고시설', '판매및영업시설', '판매시설', '05~06',
       '06~07', '07~08', '08~09', '09~10', '10~11', '11~12', '12~13', '13~14',
       '14~15', '15~16', '16~17', '17~18', '18~19', '19~20', '20~21', '21~22',
       '22~23', '23~24', '2019승객수', '2020총수송인구수', 'cluster'],
      dtype='object')

In [24]:
x_train = train_data.drop(["역명",'2020총수송인구수','2019승객수', 'cluster'], axis=1)

In [25]:
y_train = train_data[["2020총수송인구수"]]

In [26]:
x_test = test_data.drop(["역명",'2020총수송인구수','2019승객수', 'cluster'], axis=1)

In [27]:
y_test = test_data[["2020총수송인구수"]]

In [28]:
x_test.columns

Index(['Month', '가설건축물', '공공용시설', '공동주택', '공장', '관광휴게시설', '교육연구및복지시설',
       '교육연구시설', '교정및군사시설', '노유자시설', '단독주택', '동.식물관련시설', '묘지관련시설', '문화및집회시설',
       '발전시설', '방송통신시설', '분뇨.쓰레기처리시설', '수련시설', '숙박시설', '업무시설', '운동시설', '운수시설',
       '위락시설', '위험물저장및처리시설', '의료시설', '자동차관련시설', '자원순환관련시설', '장례시설',
       '제1종근린생활시설', '제2종근린생활시설', '종교시설', '창고시설', '판매및영업시설', '판매시설', '05~06',
       '06~07', '07~08', '08~09', '09~10', '10~11', '11~12', '12~13', '13~14',
       '14~15', '15~16', '16~17', '17~18', '18~19', '19~20', '20~21', '21~22',
       '22~23', '23~24'],
      dtype='object')

In [29]:
y_test.columns

Index(['2020총수송인구수'], dtype='object')

## GBT 분석

In [30]:
#블랙박스 모형으로 모델링
from sklearn.ensemble import GradientBoostingRegressor
m1 = GradientBoostingRegressor()
m1.fit(x_train,y_train)

/Users/um/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [31]:
m1.score(x_test,y_test)

0.6240673203783532

In [32]:
#블랙박스 모형의 예측값 출력
pred = m1.predict(x_train)

In [33]:
#global surrogate model 학습
from sklearn.tree import DecisionTreeRegressor, export_graphviz
m2 = DecisionTreeRegressor(max_depth=3)
m2.fit(x_train, pred) #실제값으로 지도학습 하는 것이 아니라, m1(블랙박스 모형)의 예측값으로 지도학습 진행

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=3,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [42]:
model = GradientBoostingRegressor(min_samples_split=65, max_depth=8)

In [43]:
model.fit(x_train,y_train)

/Users/um/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=8,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=65,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [44]:
print("GBT 모델")
print("------------------------------")
print("훈련 데이터셋")
print('score = ', model.score(x_train, y_train))
train_prediction = model.predict(x_train)
print('Mean_Squared_Error = ', mean_squared_error(train_prediction, y_train))
print('RMSE = ', mean_squared_error(train_prediction, y_train)**0.5)
print("------------------------------")
print("테스트 데이터셋 성능")
print('score = ', model.score(x_test, y_test))
test_prediction = model.predict(x_test)
print('Mean_Squared_Error = ', mean_squared_error(test_prediction, y_test))
print('RMSE = ', mean_squared_error(test_prediction, y_test)**0.5)

GBT 모델
------------------------------
훈련 데이터셋
score =  0.9990598216140185
Mean_Squared_Error =  7298701.636088309
RMSE =  2701.610933515096
------------------------------
테스트 데이터셋 성능
score =  0.7689253500682652
Mean_Squared_Error =  10630024856.20166
RMSE =  103102.01189211421


In [45]:
y_test2 = y_test["2020총수송인구수"]

In [46]:
mae = mean_absolute_error(y_test2, model.predict(x_test))
print("MAE : ", mae)

MAE :  54208.95373470751


In [47]:
len(model.predict(x_test))

240

In [49]:
y1 = pd.DataFrame(model.predict(x_test).tolist())

In [50]:
y = y_test

In [51]:
len(y1)

240

In [52]:
x1 = test_data.reset_index(drop=True)

In [53]:
result = pd.concat([x1, y1], axis=1) 

In [54]:
result.to_excel("인구추정결과.xlsx")

## SVR 분석

In [55]:
from sklearn.svm import SVR

In [56]:
model = SVR(kernel='linear',gamma='auto')

In [61]:
model.fit(x_train, y_train)

/Users/um/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
    kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [62]:
print("SVR 모델")
print("------------------------------")
print("훈련 데이터셋")
print('score = ', model.score(x_train, y_train))
train_prediction = model.predict(x_train)
print('Mean_Squared_Error = ', mean_squared_error(train_prediction, y_train))
print('RMSE = ', mean_squared_error(train_prediction, y_train)**0.5)
print("------------------------------")
print("테스트 데이터셋 성능")
print('score = ', model.score(x_test, y_test))
test_prediction = model.predict(x_test)
print('Mean_Squared_Error = ', mean_squared_error(test_prediction, y_test))
print('RMSE = ', mean_squared_error(test_prediction, y_test)**0.5)

SVR 모델
------------------------------
훈련 데이터셋
score =  0.7698043059133943
Mean_Squared_Error =  1787032880.251154
RMSE =  42273.31167830543
------------------------------
테스트 데이터셋 성능
score =  0.6132987472806124
Mean_Squared_Error =  17789246589.990715
RMSE =  133376.33444502333


In [63]:
y_test2 = y_test["2020총수송인구수"]
mae = mean_absolute_error(y_test2, model.predict(x_test))
print("MAE : ", mae)

MAE :  77247.89087262894
